In [6]:
from s3prl.downstream.mdd.dataset import L2ArcticDataset
from s3prl.downstream.specaug import SpecAug
from s3prl.downstream.augment_utils.spectrogram import Spectrogram
from IPython.display import Audio, display
from torch.utils.data import RandomSampler, DataLoader, Subset
import torch
import einops


In [7]:
dataset = L2ArcticDataset(
    'train',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0',
    '/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/'
)

Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0209.TextGrid
Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0272.TextGrid


In [8]:
def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()
  display(Audio(waveform, rate=sample_rate))

In [9]:
play_audio(dataset[0][0], 44100)

In [10]:
SAMPLE_RATE = 44100
n_fft = 512
n_stft = int((n_fft // 2) + 1)
specaug = SpecAug(
    **{
        "adaptive": False,
        "adaptive_number_ratio": 0.04,
        "adaptive_size_ratio": 0.04,
        "max_n_time_masks": 20,
        "apply_time_warp": True,
        "apply_time_mask": True,
        "apply_freq_mask": True,
        "time_warp_window": 5,
        "time_mask_width_range": [0, 40],
        "freq_mask_width_range": [0, 50],
        "num_freq_mask": 4,
        "num_time_mask": 2,
    }
)
# transform = torchaudio.transforms.Spectrogram(n_fft=n_fft)
# inverse_transform = torchaudio.transforms.InverseSpectrogram(n_fft=n_fft)
# spectrogram = transform(dataset[0][0])
# print(spectrogram.dtype)
# augmented = specaug([spectrogram])
# display(augmented[0][0].size())
# display(augmented[0])
# raw = inverse_transform(spectrogram.type(torch.complex64))

device = 'cuda'

transform = Spectrogram(num_stft_frequencies=n_fft)
num_samples=10
subset = Subset(dataset, torch.arange(num_samples))
wavs = [data[0] for data in subset]
aug_specs = [specaug(einops.rearrange(transform.wav_to_spec(wav), "h w complex -> complex h w"))[0] for wav in wavs]
# flat_spectrograms  = [spec[:, :, i] for spec in spectrograms for i in range(2)]
# aug_specs, _ = specaug(flat_spectrograms)
aug_wavs = [transform.spec_to_wav(torch.complex(specs[0], specs[1])) for specs in aug_specs]
# aug_wavs = []
# for specs in aug_specs:
#     aug_wavs.append(
#         transform.spec_to_wav(torch.complex(specs[0], specs[1])) 
#     )
# spectrogram = transformed_wavs = [
#     transform.spec_to_wav(specaug(transform.wav_to_spec([dataset[0][0], dataset[1][0]])
# print(spectrogram.size())
# augmented = [specaug([spectrogram[:, :, 0]])[0][0], specaug([spectrogram[:, :, 1]])[0][0]]
# display(augmented[0][0].size())
# display(augmented[0])
# raw = transform.spec_to_wav(torch.complex(spectrogram[:, :, 0], spectrogram[:, :, 1]))
# raw = transform.spec_to_wav(torch.complex(augmented[0], augmented[1]))

# display(raw.shape)
play_audio(aug_wavs[3].cpu(), 44100)

/home/xt0r3-user/anaconda3/envs/s3prl/lib/python3.8/site-packages/torch/functional.py:650: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:863.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/home/xt0r3-user/anaconda3/envs/s3prl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
SAMPLE_RATE = 44100
n_fft = 2048
n_stft = int((n_fft // 2) + 1)
specaug = SpecAug(
    **{
        "adaptive": False,
        "adaptive_number_ratio": 0.04,
        "adaptive_size_ratio": 0.04,
        "max_n_time_masks": 20,
        "apply_time_warp": True,
        "apply_time_mask": True,
        "apply_freq_mask": True,
        "time_warp_window": 5,
        "time_mask_width_range": [0, 40],
        "freq_mask_width_range": [0, 5],
        "num_freq_mask": 4,
        "num_time_mask": 2,
    }
)
# transform = torchaudio.transforms.Spectrogram(n_fft=n_fft)
# inverse_transform = torchaudio.transforms.InverseSpectrogram(n_fft=n_fft)
# spectrogram = transform(dataset[0][0])
# print(spectrogram.dtype)
# augmented = specaug([spectrogram])
# display(augmented[0][0].size())
# display(augmented[0])
# raw = inverse_transform(spectrogram.type(torch.complex64))

device = "cuda"

transform = Spectrogram(num_stft_frequencies=n_fft)
spectrogram = transform.wav_to_spec(dataset[0][0])
print(spectrogram.size())
augmented = [specaug([spectrogram[:, :, 0]])[0][0], specaug([spectrogram[:, :, 1]])[0][0]]
display(augmented[0][0].size())
display(augmented[0])
# raw = transform.spec_to_wav(torch.complex(spectrogram[:, :, 0], spectrogram[:, :, 1]))
raw = transform.spec_to_wav(torch.complex(augmented[0], augmented[1]))

# display(raw.shape)
play_audio(raw, 44100)

torch.Size([1025, 424, 2])


torch.Size([424])

tensor([[ 1.8186e+00, -5.6271e-01, -7.1650e-01,  ..., -2.9961e+00,
          8.4421e-01, -9.9275e-02],
        [ 2.6198e+00, -1.2844e+00, -1.2278e+00,  ..., -5.9213e+00,
          6.4493e+00,  1.3069e+00],
        [ 5.3814e+00, -3.8311e+00,  1.4071e+00,  ..., -2.8252e+00,
          2.7672e-01, -8.9798e-01],
        ...,
        [ 1.1742e-02, -1.6552e-02,  6.4850e-03,  ..., -1.9935e-02,
          2.7212e-02, -8.4154e-03],
        [ 1.1829e-02, -1.4451e-02,  2.9490e-04,  ..., -8.9265e-03,
          1.2721e-02,  3.4576e-03],
        [ 1.8978e-02, -5.5631e-04,  1.0340e-02,  ...,  1.3813e-02,
          4.1886e-02,  4.3071e-02]])